# Movie Recommendation System

В этом проекте будет построена система рекомендации фильмов на основе предпочтений пользователя.

Описание датасетов:

movies.csv
- movieId (ID фильма)
- title (Название фильма)
- genres (Жанры фильма)

rating.csv
- userId (ID пользователя)
- movieId (ID фильма)
- rating (рейтинг фильма)
- timestamp (время оценки фильма)

In [2]:
#Импорт необходимых библиотек.
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Загрузка датафрейма. 
df = pd.read_csv('movies.csv')

In [3]:
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


Для дальнейшей корректной работы модели необходимо обработать колонку с наименованиями фильмов.

In [3]:
# Функция удаляет из строк все символы кроме букв, цифр и пробелов.
def cleaner_title(title):
    title_name = list([x for x in title if x.isalpha() or x.isnumeric() or x == ' '])
    title_name = "".join(title_name)
    return title_name

In [45]:
df['clean_title'] = df.title.apply(lambda x: cleaner_title(x))

In [46]:
df

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


Для работы с текстом будет использоваться TfidfVectorizer.
При создании матрицы в параметр ngram_range передадим (1, 2). Теперь, вместо того, чтобы смотреть только на отдельные слова в заголовке, этот параметр будет смотреть на два соседних слова, что делает наш поиск более точным.

In [129]:
tfidf = TfidfVectorizer(ngram_range = (1, 2))
result = tfidf.fit_transform(df.clean_title)

In [130]:
print(result)

  (0, 138185)	0.5609151642422612
  (0, 153659)	0.5236464902527855
  (0, 765)	0.2947573407787223
  (0, 138139)	0.30818287987354687
  (0, 153651)	0.4788631896261391
  (1, 76504)	0.679914841526996
  (1, 76503)	0.6556226145512709
  (1, 765)	0.3284429867728573
  (2, 93349)	0.4587178998289233
  (2, 107111)	0.4026827592738571
  (2, 61526)	0.4587178998289233
  (2, 93316)	0.2658829644982531
  (2, 107056)	0.2945915056134832
  (2, 61525)	0.4587178998289233
  (2, 765)	0.22159051090518359
  (3, 47845)	0.4482553482876628
  (3, 152011)	0.4482553482876628
  (3, 161443)	0.4482553482876628
  (3, 47844)	0.4482553482876628
  (3, 151842)	0.1883000782500215
  (3, 161425)	0.33752574781287953
  (3, 765)	0.21653641961669368
  (4, 70048)	0.39452077294643884
  (4, 111110)	0.3080717668800027
  (4, 20699)	0.4091386155137103
  :	:
  (62419, 135199)	0.335141385640017
  (62419, 846)	0.21664294220561653
  (62419, 165196)	0.3842738783112516
  (62419, 106499)	0.1903879435867097
  (62419, 143808)	0.09431759289399541
  (6

In [89]:
tfidf.vocabulary_

{'toy': 153651,
 'story': 138139,
 '1995': 765,
 'toy story': 153659,
 'story 1995': 138185,
 'jumanji': 76503,
 'jumanji 1995': 76504,
 'grumpier': 61525,
 'old': 107056,
 'men': 93316,
 'grumpier old': 61526,
 'old men': 107111,
 'men 1995': 93349,
 'waiting': 161425,
 'to': 151842,
 'exhale': 47844,
 'waiting to': 161443,
 'to exhale': 152011,
 'exhale 1995': 47845,
 'father': 49667,
 'of': 104476,
 'the': 143808,
 'bride': 20653,
 'part': 111076,
 'ii': 70020,
 'father of': 49714,
 'of the': 106499,
 'the bride': 144563,
 'bride part': 20699,
 'part ii': 111110,
 'ii 1995': 70048,
 'heat': 64722,
 'heat 1995': 64739,
 'sabrina': 125338,
 'sabrina 1995': 125340,
 'tom': 152720,
 'and': 6820,
 'huck': 68768,
 'tom and': 152726,
 'and huck': 7249,
 'huck 1995': 68769,
 'sudden': 139375,
 'death': 36831,
 'sudden death': 139377,
 'death 1995': 36866,
 'goldeneye': 59719,
 'goldeneye 1995': 59721,
 'american': 6007,
 'president': 116430,
 'american president': 6141,
 'president the': 11

Для вычесления сходства между поисковым запросом и названиями фильмов будет использоваться cosine_similarity.
Для поиска написана функция которая очищает заголовок, преобразует заголовок в набор цифр, с помощью cosine_similarity находит сходство между поисковым запросом пользователя и всеми названиями в данных,
возвращает пять наиболее похожих названий по вашему поисковому запросу.

In [195]:
def search(name):
    title = cleaner_title(name)
    query_vec = tfidf.transform([title])
    similarity = cosine_similarity(query_vec, result).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = df.iloc[indices].sort_values('movieId')
    return results

Следующим шагом необходимо создать интерактивный виджет Jupyter Notebook, в котором можно ввести название фильма и просмотреть результаты поиска.

In [198]:
movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

Создадим функцию для поиска всех пользователей, которым понравился тот же фильм. 
Затем необходимо посмотреть другие фильмы, которые им понравились, так как они, возможно, будут хорошими рекомендациями.

In [201]:
rating_movie = pd.read_csv('ratings.csv')

In [202]:
rating_movie

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


Найдем пользователей, которым понравился тот же фильм. Затем нам нужно найти другие фильмы, которые эти пользователи высоко оценили. После чего установим пороговое значение для рекомендаций. Например, мы могли бы сказать, что по крайней мере 10% таких пользователей, как мы, должны поставить лайк фильму для включения в наши рекомендации.

In [212]:
similar_users = rating_movie[(rating_movie.movieId==movie_id) & (rating_movie.rating > 4)]['userId'].unique()

In [213]:
similar_users

array([    36,     75,     86, ..., 162527, 162530, 162533], dtype=int64)

In [234]:
similar_user_rec = rating_movie[(rating_movie.userId.isin(similar_users)) & (rating_movie.rating > 4)]['movieId']

In [235]:
similar_user_rec = similar_user_rec.value_counts() / len(similar_users)
similar_user_rec = similar_user_rec[similar_user_rec > .1]

In [245]:
similar_user_rec

1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: movieId, Length: 113, dtype: float64

In [248]:
all_users = rating_movie[(rating_movie.movieId.isin(similar_user_rec.index)) & (rating_movie.rating > 4)]

In [249]:
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000062,162541,5618,4.5,1240953299
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613
25000081,162541,7361,4.5,1240953484


In [250]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [251]:
all_user_recs

318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: movieId, Length: 113, dtype: float64

Now that we found the percentages, we compare them.

In [252]:
rec_percentages = pd.concat([similar_user_rec, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [256]:
rec_percentages

,similar,all,score
1,1.000000,0.124728,8.017414
32,0.160711,0.100293,1.602424
34,0.130555,0.052229,2.499660
47,0.225909,0.144469,1.563719
50,0.275604,0.200513,1.374497
...,...,...,...
59315,0.104593,0.054269,1.927310
60069,0.170640,0.076307,2.236221
68954,0.159172,0.064944,2.450924
78499,0.152960,0.035131,4.354038


In [255]:
rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']

In [257]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [258]:
rec_percentages.head(10).merge(df, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


Now we need to put all of these into a function. It should return the following columns of our top 10 movie recommendations

In [266]:
def find_similar_movies(movie_id):
    similar_users = rating_movie[(rating_movie.movieId==movie_id) & (rating_movie.rating > 4)]['userId'].unique()
    similar_user_rec = rating_movie[(rating_movie.userId.isin(similar_users)) & (rating_movie.rating > 4)]['movieId']
    
    similar_user_rec = similar_user_rec.value_counts() / len(similar_users)
    similar_user_rec = similar_user_rec[similar_user_rec > .1]

    all_users = rating_movie[(rating_movie.movieId.isin(similar_user_rec.index)) & (rating_movie.rating > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_rec, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']
    
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(df, left_index=True, right_on="movieId")[['score', 'title', 'genres']]

Now we can build the widget that will do this automatically so we can type in a movie title and get recommendations.

In [268]:
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()